In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchmetrics
from typing import Optional
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from nn import *
from preprocess import *
from kfold import *
from train import train_one_epoch
from imblearn.over_sampling import SMOTE
from collections import Counter
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


/home/estsoft/miniconda3/envs/MathAI/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#device = torch.device(args.device)
#train_df2 = pd.read_csv('../seoik/jung.csv')

train_df = pd.read_csv('data/processed.csv')
train_df['도광역시'].value_counts()
#X = get_X(train_df)
#y = get_y(train_df)[:,np.newaxis]

train_df = train_df[train_df['도광역시'] != '미분류']
train_df['도광역시'].value_counts()

#train_df.to_csv('data/final.csv', index=False)

도광역시
서울특별시      9665
경기도        7665
대구광역시      2361
경상북도       1563
경상남도       1349
부산광역시      1094
충청북도        948
전라남도        834
대전광역시       748
광주광역시       745
인천광역시       686
충청남도        658
전라북도        652
울산광역시       560
강원도         400
제주특별자치도     322
세종특별자치시     122
Name: count, dtype: int64

In [3]:
#X = get_X(train_df)
#y = get_y(train_df)[:,np.newaxis]






In [4]:
train_df['노면상태'].value_counts()

노면상태
건조       29010
젖음/습기      990
기타         334
서리/결빙       21
적설          15
침수           2
Name: count, dtype: int64

In [5]:
value_to_drop = ['기타']
column_to_check = ['노면상태','기상상태']

# Create a boolean mask for rows with the specified value in the specified column
mask = train_df[column_to_check].isin(value_to_drop).any(axis=1)

# Drop rows based on the mask
df = train_df[~mask]

# If you want to reset the index after dropping rows
df.reset_index(drop=True, inplace=True)

df['기상상태'].value_counts()

#df.to_csv('data/final.csv', index=False)

기상상태
맑음    28468
흐림      720
비       670
눈        31
안개        4
Name: count, dtype: int64

In [6]:
df['월'] = df['월'].astype(object)
#df.to_csv('data/final.csv', index=False)
df.info()

print(torch.cuda.is_available())

/tmp/ipykernel_371803/188592653.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['월'] = df['월'].astype(object)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29893 entries, 0 to 29892
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   요일          29893 non-null  object
 1   사고내용        29893 non-null  object
 2   사망자수        29893 non-null  int64 
 3   중상자수        29893 non-null  int64 
 4   경상자수        29893 non-null  int64 
 5   부상신고자수      29893 non-null  int64 
 6   사고유형        29893 non-null  object
 7   법규위반        29893 non-null  object
 8   노면상태        29893 non-null  object
 9   기상상태        29893 non-null  object
 10  도로형태        29893 non-null  object
 11  가해운전자 차종    29893 non-null  object
 12  가해운전자 성별    29893 non-null  object
 13  가해운전자 연령    29893 non-null  int64 
 14  피해운전자 차종    29893 non-null  object
 15  피해운전자 성별    29893 non-null  object
 16  피해운전자 연령    29893 non-null  int64 
 17  피해운전자 상해정도  29893 non-null  object
 18  사고시         29893 non-null  int64 
 19  월           29893 non-null  object
 20  도광역시  

In [7]:
X = get_X(df, ['요일','사고유형','법규위반','노면상태',
                                          '기상상태','도로형태','가해운전자 성별',
                                          '가해운전자 연령','피해운전자 차종','피해운전자 성별',
                                          '피해운전자 연령','피해운전자 상해정도','월',
                                          '도광역시','하루시간구분'])
y = get_y(df)[:,np.newaxis]

device = torch.device('cpu')
#print(X.shape,y.shape)

X_trn, X_test, y_trn, y_test = split2(X,y)

#SMOTE 사용하기
sm = SMOTE(random_state=4)
X_trn, y_trn = sm.fit_resample(X_trn, y_trn)
print('Resampled dataset shape %s' % Counter(y_trn))

X_trn, X_test, y_trn, y_test = torch.tensor(X_trn), torch.tensor(X_test),torch.tensor(y_trn), torch.tensor(y_test)
X_trn, X_test, y_trn, y_test = X_trn.to(device), X_test.to(device), y_trn.to(device) , y_test.to(device)


print(X_trn.shape, y_trn.shape , X_trn.dtype, y_trn.dtype)


Resampled dataset shape Counter({1.0: 12701, 0.0: 12701, 2.0: 12701})
torch.Size([38103, 105]) torch.Size([38103]) torch.float32 torch.float32


In [8]:

#Loader 올리기
ds = TensorDataset(X_trn, y_trn.squeeze().to(torch.long))
dl = DataLoader(ds, batch_size=128, shuffle=True)


In [9]:

#모델 생성하고 그냥 train 해보기

model = ANN(X_trn.shape[-1],256).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)


In [10]:
'''
pbar = range(200)
if True:
    pbar = tqdm(pbar)
for _ in pbar:
    loss = train_one_epoch(model, nn.CrossEntropyLoss(), optimizer, dl, device)
    pbar.set_postfix(trn_loss=loss)

torch.save(model.state_dict(), "./model.pth")
'''

'\npbar = range(200)\nif True:\n    pbar = tqdm(pbar)\nfor _ in pbar:\n    loss = train_one_epoch(model, nn.CrossEntropyLoss(), optimizer, dl, device)\n    pbar.set_postfix(trn_loss=loss)\n\ntorch.save(model.state_dict(), "./model.pth")\n'

In [11]:
#X=torch.tensor(X)
#y=torch.tensor(y)
kfold_cross_validate(model,criterion=nn.CrossEntropyLoss(), device=device, X_trn=X_trn, y_trn = y_trn, n_splits=5 , lr=0.0001, epochs=200)

  0%|          | 0/200 [00:00<?, ?it/s]

 80%|████████  | 161/200 [03:58<00:57,  1.48s/it, trn_loss=0.588, val_acc=0.699, val_loss=0.592]


Early stopping at epoch 161.


 66%|██████▌   | 131/200 [03:11<01:40,  1.46s/it, trn_loss=0.593, val_acc=0.696, val_loss=0.591]


Early stopping at epoch 131.


...

Early stopping at epoch 138.


 57%|█████▋    | 114/200 [02:48<02:07,  1.48s/it, trn_loss=0.593, val_acc=0.695, val_loss=0.598]


Early stopping at epoch 114.


 60%|█████▉    | 119/200 [03:06<02:06,  1.56s/it, trn_loss=0.593, val_acc=0.695, val_loss=0.595]


Early stopping at epoch 119.


In [12]:
final = pd.read_csv("data/final.csv")

X = get_X(final)
y = get_y(final)

final.isnull().sum()
pd.DataFrame(X).isnull().sum()
pd.DataFrame(y).value_counts()

#결측은 없다.

1.0    15884
0.0     8613
2.0     5396
Name: count, dtype: int64